In [18]:
def set_paths():
    '''
    Deployment should be something like "dirs/dir1/use-cases"
    This function adds to the path "dirs/dir1/use-cases" and "dirs/dir1/"
    :return:
    '''
    import imp
    from os.path import dirname
    import os
    import sys

    USE_CASES = "/var/SP/data/home/asaezco/src/devel2/use-cases"#dirname(os.path.abspath(imp.find_module('churn')[1]))
    sys.path.append("/var/SP/data/home/asaezco/src/devel2/pykhaos")
    if USE_CASES not in sys.path:
        sys.path.append(USE_CASES)
        print("Added '{}' to path".format(USE_CASES))

    # if deployment is correct, this path should be the one that contains "use-cases", "pykhaos", ...
    # FIXME another way of doing it more general?
    DEVEL_SRC = os.path.dirname(USE_CASES)  # dir before use-cases dir
    if DEVEL_SRC not in sys.path:
        sys.path.append(DEVEL_SRC)
        print("Added '{}' to path".format(DEVEL_SRC))
set_paths()

In [19]:
# coding=utf-8

import sys

from common.src.main.python.utils.hdfs_generic import *
import argparse
import os
import sys
import time
from pyspark.sql.functions import (udf,
                                    col,
                                    decode,
                                    when,
                                    lit,
                                    lower,
                                    concat,
                                    translate,
                                    count,
                                    sum as sql_sum,
                                    max as sql_max,
                                    min as sql_min,
                                    avg as sql_avg,
                                    greatest,
                                    least,
                                    isnull,
                                    isnan,
                                    struct, 
                                    substring,
                                    size,
                                    length,
                                    year,
                                    month,
                                    dayofmonth,
                                    unix_timestamp,
                                    date_format,
                                    from_unixtime,
                                    datediff,
                                    to_date, 
                                    desc,
                                    asc,
                                    countDistinct,
                                    row_number,
                                    skewness,
                                    kurtosis,
                                    concat_ws)

from pyspark.sql import Row, DataFrame, Column, Window
from pyspark.sql.types import DoubleType, StringType, IntegerType, DateType, ArrayType
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer, VectorAssembler, SQLTransformer, OneHotEncoder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from datetime import datetime
from itertools import chain
import numpy as np
from functools import reduce
from utils_general import *
from pykhaos.utils.date_functions import convert_to_date
from utils_model import *
from utils_fbb_churn import *
from metadata_fbb_churn import *
from feature_selection_utils import *
import subprocess


In [20]:
# Para que no haga falta recargar el notebook si hacemos un cambio en alguna de las funciones que estamos importando
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
# set BDP parameters
def setting_bdp(min_n_executors = 3, max_n_executors = 15, n_cores = 8, executor_memory = "12g", driver_memory="8g",
                   app_name = "Python app", driver_overhead="1g", executor_overhead='3g'):

    MAX_N_EXECUTORS = max_n_executors
    MIN_N_EXECUTORS = min_n_executors
    N_CORES_EXECUTOR = n_cores
    EXECUTOR_IDLE_MAX_TIME = 120
    EXECUTOR_MEMORY = executor_memory
    DRIVER_MEMORY = driver_memory
    N_CORES_DRIVER = 1
    MEMORY_OVERHEAD = N_CORES_EXECUTOR * 2048
    QUEUE = "root.BDPtenants.es.medium"
    BDA_CORE_VERSION = "1.0.0"

    SPARK_COMMON_OPTS = os.environ.get('SPARK_COMMON_OPTS', '')
    SPARK_COMMON_OPTS += " --executor-memory %s --driver-memory %s" % (EXECUTOR_MEMORY, DRIVER_MEMORY)
    SPARK_COMMON_OPTS += " --conf spark.shuffle.manager=tungsten-sort"
    SPARK_COMMON_OPTS += "  --queue %s" % QUEUE

    # Dynamic allocation configuration
    SPARK_COMMON_OPTS += " --conf spark.dynamicAllocation.enabled=true"
    SPARK_COMMON_OPTS += " --conf spark.shuffle.service.enabled=true"
    SPARK_COMMON_OPTS += " --conf spark.dynamicAllocation.maxExecutors=%s" % (MAX_N_EXECUTORS)
    SPARK_COMMON_OPTS += " --conf spark.dynamicAllocation.minExecutors=%s" % (MIN_N_EXECUTORS)
    SPARK_COMMON_OPTS += " --conf spark.executor.cores=%s" % (N_CORES_EXECUTOR)
    SPARK_COMMON_OPTS += " --conf spark.dynamicAllocation.executorIdleTimeout=%s" % (EXECUTOR_IDLE_MAX_TIME)
    # SPARK_COMMON_OPTS += " --conf spark.ui.port=58235"
    SPARK_COMMON_OPTS += " --conf spark.port.maxRetries=100"
    SPARK_COMMON_OPTS += " --conf spark.app.name='%s'" % (app_name)
    SPARK_COMMON_OPTS += " --conf spark.submit.deployMode=client"
    SPARK_COMMON_OPTS += " --conf spark.ui.showConsoleProgress=true"
    SPARK_COMMON_OPTS += " --conf spark.sql.broadcastTimeout=1200"
    SPARK_COMMON_OPTS += " --conf spark.yarn.executor.memoryOverhead={}".format(executor_overhead)
    SPARK_COMMON_OPTS += " --conf spark.yarn.executor.driverOverhead={}".format(driver_overhead)

    BDA_ENV = os.environ.get('BDA_USER_HOME', '')

    # Attach bda-core-ra codebase
    SPARK_COMMON_OPTS+=" --files {}/scripts/properties/red_agent/nodes.properties,{}/scripts/properties/red_agent/nodes-de.properties,{}/scripts/properties/red_agent/nodes-es.properties,{}/scripts/properties/red_agent/nodes-ie.properties,{}/scripts/properties/red_agent/nodes-it.properties,{}/scripts/properties/red_agent/nodes-pt.properties,{}/scripts/properties/red_agent/nodes-uk.properties".format(*[BDA_ENV]*7)

    os.environ["SPARK_COMMON_OPTS"] = SPARK_COMMON_OPTS
    os.environ["PYSPARK_SUBMIT_ARGS"] = "%s pyspark-shell " % SPARK_COMMON_OPTS

In [22]:
def get_spark_session(app_name="default name", log_level='INFO', min_n_executors = 3, max_n_executors = 15, n_cores = 4, executor_memory = "12g", driver_memory="8g"):
    HOME_SRC = os.path.join(os.environ.get('BDA_USER_HOME', ''), "src")
    if HOME_SRC not in sys.path:
        sys.path.append(HOME_SRC)


    setting_bdp(app_name=app_name, min_n_executors = min_n_executors, max_n_executors = max_n_executors, n_cores = n_cores, executor_memory = executor_memory, driver_memory=driver_memory)
    from common.src.main.python.utils.hdfs_generic import run_sc
    sc, spark, sql_context = run_sc(log_level=log_level)


    return sc, spark, sql_context

In [23]:
def initialize(app_name, min_n_executors = 3, max_n_executors = 15, n_cores = 4, executor_memory = "12g", driver_memory="8g"):
    import time
    start_time = time.time()

    print("_initialize spark")
    #import pykhaos.utils.pyspark_configuration as pyspark_config
    sc, spark, sql_context = get_spark_session(app_name=app_name, log_level="OFF", min_n_executors = min_n_executors, max_n_executors = max_n_executors, n_cores = n_cores,
                             executor_memory = executor_memory, driver_memory=driver_memory)
    print("Ended spark session: {} secs | default parallelism={}".format(time.time() - start_time,
                                                                         sc.defaultParallelism))
    return spark

In [24]:
spark = initialize("FBB Improvement ",executor_memory = "32g",min_n_executors = 6,max_n_executors = 15)


_initialize spark
Ended spark session: 190.99148798 secs | default parallelism=2


In [25]:
selcols = getIdFeats() + getCrmFeats() + getBillingFeats() + getMobSopoFeats() + getOrdersFeats()
now = datetime.now()


In [26]:
date_name = str('20190328')#str(now.year) + str(now.month).rjust(2, '0') + str(now.day).rjust(2, '0')
origin = '/user/hive/warehouse/tests_es.db/jvmm_amdocs_ids_'

In [27]:
# Fecha inicial va a ser un ciclo
# Fecha final es la fecha inicial+4 ciclos
trcycle_ini = '20181231'  #Training data
horizon = 4 #En ciclos
ttcycle_ini= '20190131' #Test data
tr_ttdates = trcycle_ini + '_' + ttcycle_ini

In [28]:
# 1. TRAINING DATA
# 1.1. Loading training data

In [29]:
inittrdf_ini = getFbbChurnLabeledCarCycles(spark, origin,trcycle_ini ,selcols, horizon)

[Info getCarNumClienteDf] Mon Jun  3 10:45:40 2019 Size of the original df for 20181231: 13683905
[Info FbbChurn] Mon Jun  3 10:53:39 2019 Saving origindf_tmp to HDFS
[Info getCarNumClienteDf] Mon Jun  3 10:53:39 2019 Size of the origindf after load from HDFS: 13683905
[Info getCarNumClienteDf] Mon Jun  3 10:55:01 2019 Size of the numclidf for 20181231 - Num rows: 2050838 - Num columns: 101
[Info FbbChurn] Mon Jun  3 10:58:33 2019 Saving numclidf_tmp to HDFS
[Info get_billing_df] Mon Jun  3 10:58:34 2019 Starting the preparation of billing feats
[Info get_billing_df] Mon Jun  3 10:59:20 2019 Size of the df with billing feats: 5825804
[Info getCarNumClienteDf] Mon Jun  3 10:59:45 2019 Size of the tmpdf for 20181231 - Num rows: 2050838 - Num columns: 134
[Info get_mobile_spinners_df] Mon Jun  3 11:00:25 2019 Port-out data loaded for 20181231 with a total of 27844643 rows and 11499038 distinct NIFs
[Info get_mobile_spinners_df] Mon Jun  3 11:00:49 2019 Timing feats computed for a total of

In [30]:
inittrdf_ini.printSchema()

root
 |-- rgu: string (nullable = true)
 |-- campo1: string (nullable = true)
 |-- msisdn: string (nullable = true)
 |-- num_cliente: string (nullable = true)
 |-- msisdn_d: string (nullable = true)
 |-- nif_cliente: string (nullable = true)
 |-- mean_dias_desde_fx_srv_basic: double (nullable = false)
 |-- movil_services: long (nullable = true)
 |-- tv_services: long (nullable = true)
 |-- total_penal_cust_pending_n3_penal_amount: double (nullable = false)
 |-- x_formato_factura: string (nullable = true)
 |-- x_user_twitter: string (nullable = true)
 |-- total_penal_cust_pending_n4_penal_amount: double (nullable = false)
 |-- dias_desde_bam-movil_fx_first: double (nullable = false)
 |-- max_dias_desde_fx_trybuy_autom_tv: double (nullable = false)
 |-- total_price_tariff: double (nullable = false)
 |-- flg_robinson: string (nullable = true)
 |-- pvr_services: double (nullable = false)
 |-- num_tariff_redl: double (nullable = false)
 |-- dias_desde_fx_fbb_upgrade: double (nullable = fals

In [31]:
##Añadimos extra features

In [32]:
## Añadimos las extra features (lo hacemos aquí fuera porque ya están filtradas por rgu)
dfExtraFeat = spark.read.parquet('/data/udf/vf_es/churn/extra_feats_mod/extra_feats/year={}/month={}/day={}'
                                            .format(int(trcycle_ini[0:4]), int(trcycle_ini[4:6]), int(trcycle_ini[6:8])))


In [33]:
dfExtraFeatfbb = dfExtraFeat.join(inittrdf_ini, ["num_cliente"], "leftsemi")


In [34]:
len(dfExtraFeatfbb.columns)

1352

In [35]:
allFeats = dfExtraFeatfbb.columns 
catCols=[item[0] for item in dfExtraFeatfbb.dtypes if item[1].startswith('string')]
numerical_feats = list(set(allFeats) - set(list(set().union(getIdFeats(),getIdFeats_tr(), getNoInputFeats(), catCols, [c + "_enc" for c in getCatFeatsCrm()], ["label"]))))


In [36]:
len(numerical_feats)

1331

In [37]:
numerical_feats

['netapps_ns_apps_web_yomvi_http_total_effective_ul_mb',
 'netapps_ns_apps_food_total_effective_ul_mb',
 'incremental_rel_delta_bill_n3_tax_amount',
 'netapps_ns_apps_web_pepephone_data_mb',
 'ccc_bucket_sub_bucket_device_delivery_repair_transfers',
 'netapps_ns_apps_facebook_timestamps',
 'incremental_abs_delta_data_per_connection',
 'ccc_bucket_sub_bucket_other_customer_information_management_customer_data',
 'navigation_masmovil_other_max_days_since_navigation',
 'incremental_rel_delta_total_connections_we',
 'netapps_ns_apps_adult_total_effective_dl_mb',
 'additional_num_tariff_redm',
 'additional_num_tariff_redl',
 'ccc_bucket_sub_bucket_churn_cancellations_transfers',
 'navigation_w4_max_count_comps',
 'netapps_ns_apps_taobao_data_mb',
 'ccc_raw_ofrecimiento',
 'ccc_bucket_sub_bucket_new_adds_process_voice_line',
 'navigation_pepephone_w1_sum_count',
 'netapps_ns_apps_tumblr_days',
 'netapps_ns_apps_dropbox_timestamps',
 'netapps_ns_apps_pets_days',
 'netapps_ns_apps_alcohol_data

In [74]:
apps = [name_ for name_ in numerical_feats if name_.startswith('netapps')]
tgs = [name_ for name_ in numerical_feats if name_.startswith('tgs')]
order = [name_ for name_ in numerical_feats if name_.startswith('orders')]
pbms = [name_ for name_ in numerical_feats if name_.startswith('pbms')] + ['others_ind_pbma_srv']
device = [name_ for name_ in numerical_feats if name_.startswith('device')]
ccc = [name_ for name_ in numerical_feats if name_.startswith('ccc')]
campaigns = [name_ for name_ in numerical_feats if name_.startswith('campaigns')]

incremental_bill = [name_ for name_ in numerical_feats if name_.startswith('incremental') and ('bill' in name_ or 'price' in name_ or 'tariff' in name_)]
incremental_penal = [name_ for name_ in numerical_feats if name_.startswith('incremental') and 'penal' in name_]
incremental_data = [name_ for name_ in numerical_feats if name_.startswith('incremental') and ('total_data' in name_ or 'max_mou' in name_ or 'max_connections' in name_ or 'data_per' in name_ or 'total_connections' in name_ or 'max_data' in name_ or 'max_num' in name_ or 'total_mou' in name_)]
incremental_services = [name_ for name_ in numerical_feats if name_.startswith('incremental') and ('services' in name_ or 'delta_num_' in name_)]

additional_bill = [name_ for name_ in numerical_feats if name_.startswith('additional') and ('bill' in name_ or 'price' in name_)] + ['additional_total_tv_total_charges']
additional_tariff = [name_ for name_ in numerical_feats if name_.startswith('additional') and ('num_tariff' in name_ )]
additional_days_since = [name_ for name_ in numerical_feats if name_.startswith('additional') and ('dias_desde' in name_ )]
additional_services = [name_ for name_ in numerical_feats if name_.startswith('additional') and ('services' in name_ )] + ['additional_num_2lins']

navigation = [name_ for name_ in numerical_feats if name_.startswith('navigation')]

In [172]:
penal_var = tgs + incremental_penal
inter_var = order + campaigns + ccc + pbms
use_var = apps + incremental_data 
bill_var = incremental_bill + additional_bill
serv_var = incremental_services + additional_services + additional_tariff
spin_var = navigation + additional_days_since

In [171]:
pbms

['pbms_srv_ind_averias',
 'pbms_srv_num_reclamaciones',
 'pbms_srv_ind_reclamaciones',
 'pbms_srv_ind_soporte',
 'pbms_srv_num_soporte_tecnico',
 'pbms_srv_ind_degrad_adsl',
 'pbms_srv_num_averias',
 'others_ind_pbma_srv']

In [173]:
aa = [penal_var, inter_var, use_var]

['tgs_days_since_f_inicio_bi_exp', 'tgs_has_discount', 'tgs_days_until_f_fin_bi_exp', 'tgs_days_since_f_inicio_bi', 'tgs_days_until_fecha_fin_dto', 'tgs_sum_ind_under_use', 'tgs_sum_ind_over_use', 'tgs_days_until_f_fin_bi', 'incremental_abs_delta_total_penal_cust_pending_n4_penal_amount', 'incremental_abs_delta_total_max_dias_hasta_penal_srv_pending_end_date', 'incremental_rel_delta_total_penal_cust_pending_n1_penal_amount', 'incremental_abs_delta_total_penal_srv_pending_n3_penal_amount', 'incremental_abs_delta_total_penal_cust_pending_n3_penal_amount', 'incremental_abs_delta_total_penal_cust_pending_n1_penal_amount', 'incremental_rel_delta_total_penal_srv_pending_n1_penal_amount', 'incremental_abs_delta_total_penal_srv_pending_n5_penal_amount', 'incremental_abs_delta_total_max_dias_hasta_penal_cust_pending_end_date', 'incremental_rel_delta_total_penal_srv_pending_n5_penal_amount', 'incremental_rel_delta_total_penal_cust_pending_n3_penal_amount', 'incremental_rel_delta_total_penal_cust

In [138]:
res_inc = [name_ for name_ in incremental if name_ not in incremental_penal + incremental_bill + incremental_data + incremental_services]

In [163]:
len(additional_services), len(additional_days_since),len(additional_tariff), len(additional), len(additional_bill)

(7, 6, 11, 39, 15)

In [164]:
res_add = [name_ for name_ in additional if name_ not in additional_bill + additional_tariff + additional_days_since + additional_services]

In [165]:
additional

['additional_num_tariff_redm',
 'additional_num_tariff_redl',
 'additional_inc_bill_n1_n4_amount_to_pay',
 'additional_num_tariff_maslineasmini',
 'additional_inc_bill_n1_n4_net',
 'additional_max_dias_desde_fx_trybuy_tv',
 'additional_inc_bill_n1_n5_net',
 'additional_max_dias_desde_fx_football_tv',
 'additional_num_tariff_unknown',
 'additional_num_2lins',
 'additional_num_tariff_otros',
 'additional_trybuy_services',
 'additional_trybuy_autom_services',
 'additional_football_services',
 'additional_max_dias_desde_fx_pvr_tv',
 'additional_inc_bill_n1_n2_net',
 'additional_inc_bill_n1_n2_amount_to_pay',
 'additional_max_dias_desde_fx_zapper_tv',
 'additional_num_tariff_xs',
 'additional_num_tariff_minim',
 'additional_total_price_trybuy_autom',
 'additional_total_price_football',
 'additional_total_price_pvr',
 'additional_num_tariff_megayuser',
 'additional_zapper_services',
 'additional_pvr_services',
 'additional_num_tariff_smart',
 'additional_inc_bill_n1_n3_net',
 'additional_max

In [166]:
res_add

[]

In [40]:
dfExtraFeatSel,selColumnas=addExtraFeats(dfExtraFeatfbb)

('[Info FbbChurn] Fri May 31 10:28:42 2019 The module for the Extra Feats has been run. Count for the table', 2050838)


In [41]:
dfillNa = fillNa(spark)
for kkey in dfillNa.keys():
    if kkey not in dfExtraFeatSel.columns:
        dfillNa.pop(kkey, None)

In [42]:
inittrdf = inittrdf_ini.join(dfExtraFeatSel, ["msisdn","num_cliente",'rgu'], how="left").na.fill(dfillNa)

In [26]:
inittrdf.count()

1978186

In [46]:
len(dfExtraFeatSel.columns),len(inittrdf_ini.columns),len(inittrdf.columns)

(92, 239, 328)

In [28]:
dfExtraFeatSel.count(),inittrdf_ini.count(),inittrdf.count()

(1978186, 1978186, 1978186)

In [47]:
[unbaltrdf, valdf] = inittrdf.randomSplit([0.7, 0.3], 1234)

In [ ]:
# 1.2. Balanced df for training

In [48]:
unbaltrdf.groupBy('label').agg(count('*')).show()

+-----+--------+
|label|count(1)|
+-----+--------+
|  0.0| 1386523|
|  1.0|   50294|
+-----+--------+



In [31]:
trdf = balance_df2(unbaltrdf, 'label')

[Info balance_df2] Dataframe has been balanced - Total number of rows is 88403


In [32]:
trdf.groupBy('label').agg(count('*')).show()

+-----+--------+
|label|count(1)|
+-----+--------+
|  0.0|   44124|
|  1.0|   44279|
+-----+--------+



In [49]:
[tr_df, tr_df_final] = unbaltrdf.randomSplit([0.8, 0.2], 1234)

In [77]:
range(0,10)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [50]:
tr_1s = tr_df.where(col('label') == 1)
tr_0s = tr_df.where(col('label') == 0)

In [51]:
num_tr_dfs = 3

In [159]:
from pyspark.sql.functions import monotonically_increasing_id

# This will return a new DF with all the columns + id
tr_0s_idx = tr_0s.withColumn("idx", monotonically_increasing_id())

In [173]:
ratio = num_1s / tr_0s_idx.count()

In [174]:
ratio

0.033004818156339634

In [163]:
i = 0
tr_0s_idx.where(((col('idx') >= i*num_1s) & (col('idx') < (i+1)*num_1s))).count()

6707

In [167]:
tr_0s_idx.where((col('idx') >= i*num_1s)).count()

1341592

In [168]:
tr_0s_idx.where((col('idx') < (i+1)*num_1s)).count()

6707

In [ ]:
tr_0s_idx.where((col('idx') >= i*num_1s)).where((col('idx') < (i+1)*num_1s)).count()

In [170]:
(i+1)*num_1s

44279

In [182]:
aa = np.full((1, num_tr_dfs), ratio)

In [185]:
bb = np.append(aa, 1-ratio)

In [186]:
bb

array([0.03300482, 0.03300482, 0.03300482, 0.03300482, 0.03300482,
       0.96699518])

In [187]:
selected_0s = tr_0s.randomSplit(bb)

In [188]:
len(selected_0s)

6

In [189]:
selected_0s[0].count()

38791

In [190]:
selected_0s[1].count()

39080

In [195]:
type(tr_1s.orderBy(rand()).limit(12))

pyspark.sql.dataframe.DataFrame

In [ ]:
i = 1
tr_0s_idx.where(((col('idx') >= i*num_1s) & (col('idx') < (i+1)*num_1s))).count()

In [248]:
num_1s = 44279#tr_1s.count()
ratio = num_1s / 1341592.0#tr_0s_idx.count()
arr = np.ones(num_tr_dfs + 1)
vec_selected_0s = tr_0s.randomSplit(arr)

In [198]:
num_1s = 44279#tr_1s.count()
ratio = num_1s / 1341592.0#tr_0s_idx.count()
arr = np.full((1, num_tr_dfs), ratio)
arr = np.append(arr, 1 - ratio*num_tr_dfs)
vec_selected_0s = tr_0s.randomSplit(arr)

vector_train = []
print(num_1s)
for i in range(0,num_tr_dfs):
    value = vec_selected_0s[i].count()
    tr = tr_1s.orderBy(rand()).limit(value).union(vec_selected_0s[i])
    print(value)
    vector_train.append(tr) 

44279
39367
39125
38848
39046
39374


In [ ]:
len(vector_train)

In [52]:
num_1s = tr_1s.count()
ratio = num_1s / tr_0s.count()
arr = np.ones(num_tr_dfs + 1)
vec_selected_0s = tr_0s.randomSplit(arr)

vector_train = []
print(num_1s)
for i in range(0,num_tr_dfs):
    tr = tr_1s.union(vec_selected_0s[i].limit(num_1s))
    vector_train.append(tr) 

40221


In [53]:
len(vector_train)

3

In [ ]:
# ***************************************************************************************
# ***************************************************************************************
# path = "/data/udf/vf_es/churn/fbb_tmp/trdf_" + tr_ttdates
# trdf = spark.read.parquet(path)
# ***************************************************************************************
# ***************************************************************************************

In [ ]:
# 1.3. Feature selection

In [55]:
train_feats = []
for el in vector_train:
    train_feats.append(el.columns)

In [56]:
allFeats = vector_train[0].columns #Revisar allFeats !!!!
catCols=[item[0] for item in vector_train[0].dtypes if item[1].startswith('string')]
numerical_feats = list(set(allFeats) - set(list(set().union(getIdFeats(),getIdFeats_tr(), getNoInputFeats(), catCols, [c + "_enc" for c in getCatFeatsCrm()], ["label"]))))


In [207]:
vector_train = vector_train2

In [57]:
noninf_feats_vec = []
for el in vector_train:
    noninf_feats = getNonInfFeats(el, numerical_feats)
    noninf_feats_vec.append(noninf_feats)

[Info FbbChurn] 1/2 Selecting the features that add information to the model
[Info FbbChurn] 2/2 Selecting the features that add information to the model
[Info FbbChurn] 1/2 Selecting the features that add information to the model
[Info FbbChurn] 2/2 Selecting the features that add information to the model
[Info FbbChurn] 1/2 Selecting the features that add information to the model
[Info FbbChurn] 2/2 Selecting the features that add information to the model


In [ ]:
# 2. TEST DATA

In [58]:
ttdf_ini = getFbbChurnLabeledCarCycles(spark, origin,ttcycle_ini,selcols, horizon)

[Info getCarNumClienteDf] Fri May 31 11:55:57 2019 Size of the original df for 20190131: 13706789
[Info FbbChurn] Fri May 31 12:00:18 2019 Saving origindf_tmp to HDFS
[Info getCarNumClienteDf] Fri May 31 12:00:19 2019 Size of the origindf after load from HDFS: 13706789
[Info getCarNumClienteDf] Fri May 31 12:01:40 2019 Size of the numclidf for 20190131 - Num rows: 2054979 - Num columns: 101
[Info FbbChurn] Fri May 31 12:05:09 2019 Saving numclidf_tmp to HDFS
[Info get_billing_df] Fri May 31 12:05:09 2019 Starting the preparation of billing feats
[Info get_billing_df] Fri May 31 12:05:33 2019 Size of the df with billing feats: 5820445
[Info getCarNumClienteDf] Fri May 31 12:05:45 2019 Size of the tmpdf for 20190131 - Num rows: 2054979 - Num columns: 134
[Info get_mobile_spinners_df] Fri May 31 12:05:58 2019 Port-out data loaded for 20190131 with a total of 28304454 rows and 11561987 distinct NIFs
[Info get_mobile_spinners_df] Fri May 31 12:06:16 2019 Timing feats computed for a total of

In [ ]:
##Añadimos extra features

In [64]:
dfExtraFeat_tt = spark.read.parquet('/data/udf/vf_es/churn/extra_feats_mod/extra_feats/year={}/month={}/day={}'
                                            .format(int(ttcycle_ini[0:4]), int(ttcycle_ini[4:6]), int(ttcycle_ini[6:8])))


In [65]:
dfExtraFeatfbb_tt=dfExtraFeat_tt.join(ttdf_ini.select('num_cliente'),on='num_cliente',how='right')
print(dfExtraFeatfbb_tt.select('num_cliente').distinct().count(),ttdf_ini.select('num_cliente').distinct().count()) 


(2054902, 2054902)


In [66]:
dfExtraFeat_ttSel, selColumnas = addExtraFeats(dfExtraFeatfbb_tt)

('[Info FbbChurn] Fri May 31 12:26:24 2019 The module for the Extra Feats has been run. Count for the table', 2055133)


In [67]:
dfillNa = fillNa(spark)
for kkey in dfillNa.keys():
    if kkey not in dfExtraFeat_ttSel.columns:
        dfillNa.pop(kkey, None)

In [68]:
ttdf = ttdf_ini.join(dfExtraFeat_ttSel, ["msisdn", "num_cliente", 'rgu'], how="left").na.fill(dfillNa)

In [ ]:
# ***************************************************************************************
# ***************************************************************************************
#path = "/data/udf/vf_es/churn/fbb_tmp/ttdf_" + tr_ttdates
#ttdf = spark.read.parquet(path)
# ***************************************************************************************
# ***************************************************************************************

In [ ]:
# 3. MODELLING

In [70]:
vector_feats = []
for i in range(0,num_tr_dfs):
    featCols = list(set(numerical_feats) - set(noninf_feats_vec[i]))
    vector_feats.append(featCols)

In [71]:
assembler_vec = []
for i in range(0,num_tr_dfs):
    assembler = VectorAssembler(inputCols = vector_feats[i], outputCol = "features")
    assembler_vec.append(assembler)

In [72]:
classifier = RandomForestClassifier(featuresCol="features",\
        labelCol="label",\
        maxDepth=12,\
        maxBins=32,\
        minInstancesPerNode=200,\
        impurity="gini",\
        featureSubsetStrategy="sqrt",\
        subsamplingRate=0.7,\
        numTrees=800,\
        seed = 1234,)

In [73]:
pipeline_vec = []
for i in range(0,num_tr_dfs):
    pipeline = Pipeline(stages= [assembler_vec[i], classifier])
    pipeline_vec.append(pipeline)

In [75]:
models_vec = []
for i in range(0,num_tr_dfs):
    model = pipeline_vec[i].fit(vector_train[i])
    models_vec.append(model)
    print(i)

Py4JJavaError: An error occurred while calling o13201.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 15 in stage 1995.0 failed 4 times, most recent failure: Lost task 15.3 in stage 1995.0 (TID 313495, vgddp450hr.dc.sedc.internal.vodafone.com, executor 29): java.io.FileNotFoundException: File does not exist: hdfs://nameservice1/data/udf/vf_es/churn/fbb_tmp/numclidf_tmp_20181231/part-00085-776d3a54-b207-42b7-af0c-a7a0b1376094-c000.snappy.parquet
It is possible the underlying files have been updated. You can explicitly invalidate the cache in Spark by running 'REFRESH TABLE tableName' command in SQL or by recreating the Dataset/DataFrame involved.
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.org$apache$spark$sql$execution$datasources$FileScanRDD$$anon$$readCurrentFile(FileScanRDD.scala:131)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:182)
	at org.apache.spark.sql.execution.datasourc

In [ ]:
def get_calibration_function_vec(spark, model, valdf, labelcol, numpoints = 10, num = 0):
    import numpy as np
    score_name = "model_score_" + str(num)
    score_calib_name ='calib_model_score_' + str(num)
    getScore = udf(lambda prob: float(prob[1]), DoubleType())
    valpredsdf = model.transform(valdf).withColumn(score_name, getScore(col("probability")).cast(DoubleType())).select([score_name, labelcol])
    minprob = valpredsdf.select(sql_min(score_name).alias('min')).rdd.collect()[0]['min']
    maxprob = valpredsdf.select(sql_max(score_name).alias('max')).rdd.collect()[0]['max']
    numlabels = float(valpredsdf.filter(col(labelcol) == 1.0).count())
    print "[Info Calibration] Training - Num samples in the target class " + str(numlabels)
    delta = float(maxprob - minprob)/float(numpoints)
    ths = list([(minprob + i*delta) for i in list(np.arange(1,numpoints,step=1))])
    samplepoints = [(float(i), float(valpredsdf.filter((col(score_name) <= i) & (col(labelcol) == 1.0)).count())/numlabels) for i in ths]
    # samplepoints = [(float(i), valpredsdf.filter(col('model_score') <= i &).select(sql_avg(labelcol).alias('rate')).rdd.collect()[0]['rate']) for i in ths]
    for pair in samplepoints:
        print "[Info Calibration] Training " + str(pair[0]) + ", " + str(pair[1])
    sampledf = spark.createDataFrame(samplepoints, [score_name, 'target_prob'])
    sampledf.show()
    print "[Info Calibration] Samples to implement the calibration model showed above"
    ir = IsotonicRegression(featuresCol=score_name, labelCol='target_prob', predictionCol=score_calib_name)
    irmodel = ir.fit(sampledf)
    return (irmodel, samplepoints)

In [ ]:
# ***************************************************************************************
# ***************************************************************************************
# path2 = '/data/udf/vf_es/churn/fbb_tmp/valdf_' + tr_ttdates
# valdf = spark.read.parquet(path2)
# ***************************************************************************************
# ***************************************************************************************

In [ ]:
# EVALUATION
# Calibration
calibration_vec = []
for i in range(0,num_tr_dfs):
    calibmodel = get_calibration_function_vec(spark, models_vec[i], valdf, 'label', 10, i)
    calibration_vec.append(calibmodel)
    print(i)

In [218]:
getScore = udf(lambda prob: float(prob[1]), DoubleType())

In [220]:
# Train
tr_calib_vec = []
for i in range(0, num_tr_dfs):
    tr_preds_df = models_vec[i].transform(vector_train[i]).withColumn("model_score_" + str(i), getScore(col("probability")).cast(DoubleType()))
    tr_calib_preds_df = calibration_vec[i][0].transform(tr_preds_df)
    tr_calib_vec.append(tr_calib_preds_df)
    print(i)

0
1
2
3
4


In [221]:
tr_metrics_vec = []
# Train evaluation
#tr_calib_vec = []
for i in range(0, num_tr_dfs):#'calib_model_score_' + str(num)
    trPredictionAndLabels = tr_calib_vec[i].select(['calib_model_score_' + str(i), 'label']).rdd.map(lambda r: (r['calib_model_score_' + str(i)], r['label']))
    trmetrics = BinaryClassificationMetrics(trPredictionAndLabels)
    print(trmetrics.areaUnderROC)
    tr_metrics_vec.append(trmetrics)
    print(i)

0.754406933206
0
0.755106476011
1
0.753097394074
2
0.755560008586
3
0.755012309843
4


In [222]:
# Test eval
df_aux = ttdf
for i in range(0, num_tr_dfs):
    df_aux = models_vec[i].transform(df_aux).withColumn("model_score_" + str(i), getScore(col("probability")).cast(DoubleType())).drop(col('probability'))
    # tt_preds_df
    df_aux = calibration_vec[i][0].transform(df_aux.drop(col('features')).drop(col('prediction')).drop(col('rawPrediction')))
    print(i)
    
save_dir = 'tests_es.asaezco_fbb_improved_' + str(20190131)
df_aux.write.format('parquet').mode('overwrite').saveAsTable(save_dir)

0
1
2
3
4


In [223]:
df_aux.printSchema()

root
 |-- msisdn: string (nullable = true)
 |-- num_cliente: string (nullable = true)
 |-- rgu: string (nullable = true)
 |-- campo1: string (nullable = true)
 |-- msisdn_d: string (nullable = true)
 |-- nif_cliente: string (nullable = true)
 |-- mean_dias_desde_fx_srv_basic: double (nullable = false)
 |-- movil_services: long (nullable = true)
 |-- tv_services: long (nullable = true)
 |-- total_penal_cust_pending_n3_penal_amount: double (nullable = false)
 |-- x_formato_factura: string (nullable = true)
 |-- x_user_twitter: string (nullable = true)
 |-- total_penal_cust_pending_n4_penal_amount: double (nullable = false)
 |-- dias_desde_bam-movil_fx_first: double (nullable = false)
 |-- max_dias_desde_fx_trybuy_autom_tv: double (nullable = false)
 |-- total_price_tariff: double (nullable = false)
 |-- flg_robinson: string (nullable = true)
 |-- pvr_services: double (nullable = false)
 |-- num_tariff_redl: double (nullable = false)
 |-- dias_desde_fx_fbb_upgrade: double (nullable = fals

In [65]:
# Evaluation
ttPredictionAndLabels = tt_calib_preds_df.select(['calib_model_score', 'label']).rdd.map(lambda r: (r['calib_model_score'], r['label']))
ttmetrics = BinaryClassificationMetrics(ttPredictionAndLabels)

In [66]:
print("[Info FbbChurn] " + time.ctime() + " Area under ROC(tr) = " + str(trmetrics.areaUnderROC) + " - Area under ROC(tt) = " + str(ttmetrics.areaUnderROC))


[Info FbbChurn] Wed May 29 11:42:15 2019 Area under ROC(tr) = 0.760427050776 - Area under ROC(tt) = 0.714090517079


In [224]:
for i in range(0, num_tr_dfs):
    ttPredictionAndLabels = df_aux.select(['calib_model_score_' + str(i), 'label']).rdd.map(lambda r: (r['calib_model_score_' + str(i)], r['label']))
    ttmetrics = BinaryClassificationMetrics(ttPredictionAndLabels)
    #print(ttmetrics.areaUnderROC)
    print(i)
    print "Model {}: Area under ROC(tr) = {}".format(i, str(tr_metrics_vec[i].areaUnderROC))
    print "Model {}: Area under ROC(tt) = {}".format(i, str(ttmetrics.areaUnderROC))

0
Model 0: Area under ROC(tr) = 0.754406933206
Model 0: Area under ROC(tt) = 0.713514753724
1
Model 1: Area under ROC(tr) = 0.755106476011
Model 1: Area under ROC(tt) = 0.713128028314
2
Model 2: Area under ROC(tr) = 0.753097394074
Model 2: Area under ROC(tt) = 0.712410491476
3
Model 3: Area under ROC(tr) = 0.755560008586
Model 3: Area under ROC(tt) = 0.712465860634
4
Model 4: Area under ROC(tr) = 0.755012309843
Model 4: Area under ROC(tt) = 0.713019849073


In [234]:
cols_scores

['calib_model_score_0',
 'calib_model_score_1',
 'calib_model_score_2',
 'calib_model_score_3',
 'calib_model_score_4']

In [235]:
cols_scores = [name_ for name_ in df_aux.columns if name_.startswith('calib_model_')]

from pyspark.sql.functions import greatest, least, mean, sum

df_aux2 = df_aux.withColumn('maximo', greatest(*cols_scores)).withColumn('minimo', least(*cols_scores))\
.withColumn('medio', (df_aux['calib_model_score_0'] + df_aux['calib_model_score_1'] + df_aux['calib_model_score_2'] + df_aux['calib_model_score_3'] + df_aux['calib_model_score_4'])/5)#.withColumn('minimo', least(*cols_scores))

In [229]:
for name in ['maximo','minimo']:
    tt_ensemble = df_aux2.select([name, 'label']).rdd.map(lambda r: (r[name], r['label']))
    ttmetrics = BinaryClassificationMetrics(tt_ensemble)
    print "Model {}: Area under ROC(tt) = {}".format(i, str(ttmetrics.areaUnderROC))

Model 4: Area under ROC(tt) = 0.712157131246
Model 4: Area under ROC(tt) = 0.713841078181


In [156]:
for i in range(0,2):
    print(tr_metrics_vec[i].areaUnderROC)

0.767949194081
1.0


In [236]:
for name in ['medio']:
    tt_ensemble = df_aux2.select([name, 'label']).rdd.map(lambda r: (r[name], r['label']))
    ttmetrics = BinaryClassificationMetrics(tt_ensemble)
    print "Model {}: Area under ROC(tt) = {}".format(i, str(ttmetrics.areaUnderROC))

Model 4: Area under ROC(tt) = 0.713220048003


In [249]:
tr = tr_1s.union(vec_selected_0s[num_tr_dfs].limit(num_1s))

In [250]:
tr.count()

Py4JJavaError: An error occurred while calling o32030.count.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 37 in stage 29996.0 failed 4 times, most recent failure: Lost task 37.3 in stage 29996.0 (TID 4397224, vgddp601hr.dc.sedc.internal.vodafone.com, executor 364): java.io.FileNotFoundException: File does not exist: hdfs://nameservice1/data/udf/vf_es/churn/fbb_tmp/numclidf_tmp_20181130/part-00115-96c44731-02fb-4b41-bb45-ac2d0396c457-c000.snappy.parquet
It is possible the underlying files have been updated. You can explicitly invalidate the cache in Spark by running 'REFRESH TABLE tableName' command in SQL or by recreating the Dataset/DataFrame involved.
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.org$apache$spark$sql$execution$datasources$FileScanRDD$$anon$$readCurrentFile(FileScanRDD.scala:131)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:182)
	at org.apache.spark.sql.execution.dat

In [238]:
df_aux_tr = tr
for i in range(0, num_tr_dfs):
    df_aux_tr = models_vec[i].transform(df_aux_tr).withColumn("model_score_" + str(i), getScore(col("probability")).cast(DoubleType())).drop(col('probability'))
    # tt_preds_df
    df_aux_tr = calibration_vec[i][0].transform(df_aux_tr.drop(col('features')).drop(col('prediction')).drop(col('rawPrediction')))
    print(i)

0
1
2
3
4


In [239]:
features_ = cols_scores
assembler = VectorAssembler(inputCols = features_, outputCol = "features")

classifier = RandomForestClassifier(featuresCol="features",\
        labelCol="label",\
        maxDepth=10,\
        maxBins=32,\
        minInstancesPerNode=200,\
        impurity="gini",\
        featureSubsetStrategy="sqrt",\
        subsamplingRate=0.7,\
        numTrees=800,\
        seed = 1234,)

pipeline = Pipeline(stages= [assembler, classifier])

In [243]:
tr.count()

Py4JJavaError: An error occurred while calling o31302.count.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 8 in stage 29896.0 failed 4 times, most recent failure: Lost task 8.3 in stage 29896.0 (TID 4397001, vgddp426hr.dc.sedc.internal.vodafone.com, executor 361): java.io.FileNotFoundException: File does not exist: hdfs://nameservice1/data/udf/vf_es/churn/fbb_tmp/numclidf_tmp_20181130/part-00175-96c44731-02fb-4b41-bb45-ac2d0396c457-c000.snappy.parquet
It is possible the underlying files have been updated. You can explicitly invalidate the cache in Spark by running 'REFRESH TABLE tableName' command in SQL or by recreating the Dataset/DataFrame involved.
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.org$apache$spark$sql$execution$datasources$FileScanRDD$$anon$$readCurrentFile(FileScanRDD.scala:131)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:182)
	at org.apache.spark.sql.execution.datas

In [240]:
model_final = pipeline.fit(df_aux_tr)

Py4JJavaError: An error occurred while calling o31774.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 8 in stage 29790.0 failed 4 times, most recent failure: Lost task 8.3 in stage 29790.0 (TID 4396812, vgddp426hr.dc.sedc.internal.vodafone.com, executor 361): java.io.FileNotFoundException: File does not exist: hdfs://nameservice1/data/udf/vf_es/churn/fbb_tmp/numclidf_tmp_20181130/part-00175-96c44731-02fb-4b41-bb45-ac2d0396c457-c000.snappy.parquet
It is possible the underlying files have been updated. You can explicitly invalidate the cache in Spark by running 'REFRESH TABLE tableName' command in SQL or by recreating the Dataset/DataFrame involved.
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.org$apache$spark$sql$execution$datasources$FileScanRDD$$anon$$readCurrentFile(FileScanRDD.scala:131)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:182)
	at org.apache.spark.sql.execution.datasou

In [ ]:
.transform(tr).withColumn("model_score", getScore(col("probability")).cast(DoubleType()))